Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

# Section 1

In [234]:
# import the library we use to open URLs
import urllib.request

In [235]:
!conda install -c conda-forge geopy --yes

In [236]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [237]:
import lxml.html as lh
import requests
import pandas as pd
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
#Store the contents of the website
data = lh.fromstring(page.content)
#Store table data
tr_elements = data.xpath('//tr')
col=[]
ind=0
#store each first element and an empty list for each row
for i in tr_elements[0]:
    ind= ind + 1
    name = i.text_content()
    col.append((name,[]))
#Since the first row is header, data is stored from the second row
for i in range(1,len(tr_elements)):
    T=tr_elements[i]
    
    #If row is not of size 3, then tr data is not from our table 
    if len(T)!=3:
        break
    #i is the index of our column
    ind=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        temp=t.text_content() 
        #Check if row is empty
        if ind>0:
        #Convert any numerical value to integers
            try:
                temp=int(temp)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[ind][1].append(temp)
        #Increment i for the next column
        ind = ind + 1
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postal Code\n,Borough\n,Neighbourhood\n
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [238]:
df['Postal Code\n']=df['Postal Code\n'].str.replace('\n', '')

In [239]:
df['Borough\n']=df['Borough\n'].str.replace('\n', '')

In [240]:
df['Neighbourhood\n']=df['Neighbourhood\n'].str.replace('\n', '')

In [241]:
df.rename(columns={'Postal Code\n': 'Postal Code', 'Borough\n': 'Borough', 'Neighbourhood\n': 'Neighbourhood'}, inplace = True)

In [242]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


The cell below will result in the final table for 5 Neighbourhood of Toronto 

In [243]:
df = df[df.Borough != 'Not assigned']
df = df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [244]:
df.shape

(104, 3)

# Section 2

In [245]:
!conda install -c conda-forge geocoder -y

#### Use geopy library to get the latitude and longitude values of Toronto.

In [246]:
import geocoder # import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude


print(latitude_toronto,longitude_toronto )

43.6534817 -79.3839347


The code below results in long. and lat. of every neighborhood. 

In [247]:
latitude=[]
longitude=[]
for code in df['Postal Code']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

M3A [43.75245000000007, -79.32990999999998]
M4A [43.73057000000006, -79.31305999999995]
M5A [43.65512000000007, -79.36263999999994]
M6A [43.72327000000007, -79.45041999999995]
M7A [43.66253000000006, -79.39187999999996]
M9A [43.662630000000036, -79.52830999999998]
M1B [43.811390000000074, -79.19661999999994]
M3B [43.74923000000007, -79.36185999999998]
M4B [43.70718000000005, -79.31191999999999]
M5B [43.65739000000008, -79.37803999999994]
M6B [43.70687000000004, -79.44811999999996]
M9B [43.65034000000003, -79.55361999999997]
M1C [43.78574000000003, -79.15874999999994]
M3C [43.72168000000005, -79.34351999999996]
M4C [43.68970000000007, -79.30681999999996]
M5C [43.65215000000006, -79.37586999999996]
M6C [43.69211000000007, -79.43035999999995]
M9C [43.64857000000006, -79.57824999999997]
M1E [43.765750000000025, -79.17469999999997]
M4E [43.67709000000008, -79.29546999999997]
M5E [43.64536000000004, -79.37305999999995]
M6E [43.68784000000005, -79.45045999999996]
M1G [43.76812000000007, -79.2

In [248]:
df['Latitude'] = latitude
df['Longitude'] = longitude

In [249]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188


The exact latitude and longitude differ a bit from the table in the instructions due to the difference in the sources (Google vs geocoder.arcgis)

# Section 3

#### Create a map of Toronto with neighborhoods superimposed on top.

In [250]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare Credentials and Version

In [251]:
CLIENT_ID = 'ZLZFO2UF3SV0BCZ5YLCNB43VVORVUCV2TJMBNMRCC0S3OWPZ' # your Foursquare ID
CLIENT_SECRET = 'ECHBDUNLVFS4U2X4OLDSHUMJYO0F1AMCBXUY2ICBIELWDE2R' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZLZFO2UF3SV0BCZ5YLCNB43VVORVUCV2TJMBNMRCC0S3OWPZ
CLIENT_SECRET:ECHBDUNLVFS4U2X4OLDSHUMJYO0F1AMCBXUY2ICBIELWDE2R


## Explore Neighborhoods in Parkwoods, Toronto

I decided to explore all the neighborhoods in Toronto

In [252]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [253]:
all_toronto_venues = getNearbyVenues(df['Neighbourhood'], df['Latitude'], df['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [254]:
all_toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75245,-79.32991,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75245,-79.32991,Brookbanks Pool,43.751389,-79.332184,Pool
2,Parkwoods,43.75245,-79.32991,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.73057,-79.31306,Wigmore Park,43.731023,-79.310771,Park
4,Victoria Village,43.73057,-79.31306,Memories of Africa,43.726602,-79.312427,Grocery Store


## Analyze Each Neighborhood

In [255]:
# one hot encoding
all_toronto_onehot = pd.get_dummies(all_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
all_toronto_onehot['Neighborhood'] = all_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [all_toronto_onehot.columns[-1]] + list(all_toronto_onehot.columns[:-1])
all_toronto_onehot = all_toronto_onehot[fixed_columns]

all_toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [256]:
all_toronto_grouped = all_toronto_onehot.groupby('Neighborhood').mean().reset_index()
all_toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.030000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.01

#### Let's print each neighborhood along with the top 5 most common venues

In [257]:
num_top_venues = 5

for hood in all_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = all_toronto_grouped[all_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

--------
              venue  freq
0       Coffee Shop  0.07
1             Hotel  0.05
2              Café  0.04
3        Restaurant  0.03
4  Asian Restaurant  0.03


----Agincourt----
                venue  freq
0  Chinese Restaurant  0.15
1       Shopping Mall  0.08
2    Sushi Restaurant  0.08
3              Bakery  0.08
4         Supermarket  0.08


----Alderwood, Long Branch----
                        venue  freq
0                         Pub  0.25
1           Convenience Store  0.25
2       Performing Arts Venue  0.25
3                         Gym  0.25
4  Modern European Restaurant  0.00


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0           Business Service   1.0
1                Yoga Studio   0.0
2                Men's Store   0.0
3         Mexican Restaurant   0.0
4  Middle Eastern Restaurant   0.0


----Bayview Village----
                        venue  freq
0                       Trail  0.50
1  Construction & Landscaping  0

In [258]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [259]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = all_toronto_grouped['Neighborhood']

for ind in np.arange(all_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(all_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,,Coffee Shop,Hotel,Café,Restaurant,Asian Restaurant,Japanese Restaurant,Concert Hall,Gym,Bar,Sushi Restaurant
1,Agincourt,Chinese Restaurant,Discount Store,Department Store,Shopping Mall,Bubble Tea Shop,Supermarket,Sushi Restaurant,Bakery,Badminton Court,Skating Rink
2,"Alderwood, Long Branch",Performing Arts Venue,Gym,Convenience Store,Pub,Women's Store,Ethiopian Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School
3,"Bathurst Manor, Wilson Heights, Downsview North",Business Service,Women's Store,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
4,Bayview Village,Trail,Construction & Landscaping,Park,Women's Store,Ethiopian Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room


## Cluster Neighborhoods

In [298]:
# set number of clusters
kclusters = 5

all_toronto_grouped_clustering = all_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(all_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 4, 2, 3, 3, 3, 3, 3], dtype=int32)

In [299]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

all_toronto_merged = df

# merge all_toronto_grouped with all_toronto_data to add latitude/longitude for each neighborhood
all_toronto_merged = all_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood', how = 'right')

all_toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
103,,Canadian postal codes,,43.64869,-79.38544,3,Coffee Shop,Hotel,Café,Restaurant,Asian Restaurant,Japanese Restaurant,Concert Hall,Gym,Bar,Sushi Restaurant
78,M1S,Scarborough,Agincourt,43.79452,-79.26708,3,Chinese Restaurant,Discount Store,Department Store,Shopping Mall,Bubble Tea Shop,Supermarket,Sushi Restaurant,Bakery,Badminton Court,Skating Rink
93,M8W,Etobicoke,"Alderwood, Long Branch",43.60124,-79.53879,3,Performing Arts Venue,Gym,Convenience Store,Pub,Women's Store,Ethiopian Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School
28,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.75788,-79.44847,4,Business Service,Women's Store,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
39,M2K,North York,Bayview Village,43.78112,-79.38060,2,Trail,Construction & Landscaping,Park,Women's Store,Ethiopian Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room


The cell below plots the final cluster map

In [300]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(all_toronto_merged['Latitude'], all_toronto_merged['Longitude'], all_toronto_merged['Neighbourhood'], all_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [301]:
all_toronto_merged.loc[all_toronto_merged['Cluster Labels'] == 0, all_toronto_merged.columns[[1] + list(range(5, all_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,East York,0,Home Service,Intersection,Park,Women's Store,Escape Room,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School
17,Etobicoke,0,Fish & Chips Shop,Park,Shopping Mall,Electronics Store,Grocery Store,Carpet Store,College Rec Center,Convenience Store,Dog Run,Farmers Market
18,Scarborough,0,Construction & Landscaping,Gym / Fitness Center,Park,Women's Store,Ethiopian Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room
69,West Toronto,0,Residential Building (Apartment / Condo),Sandwich Place,Park,Women's Store,Elementary School,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room
27,North York,0,Residential Building (Apartment / Condo),Park,Women's Store,Escape Room,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant
57,North York,0,Coffee Shop,Nightclub,Park,Women's Store,Escape Room,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Event Space
16,York,0,Field,Playground,Hockey Arena,Grocery Store,Park,Trail,Women's Store,Elementary School,Dog Run,Donut Shop
38,Scarborough,0,Department Store,Playground,Park,Coffee Shop,Women's Store,Escape Room,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School
83,Central Toronto,0,Gym,Park,Women's Store,Escape Room,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Event Space
88,Etobicoke,0,Park,Yoga Studio,Grocery Store,Convenience Store,Skating Rink,Tennis Court,Elementary School,Dog Run,Donut Shop,Eastern European Restaurant


In [302]:
all_toronto_merged.loc[all_toronto_merged['Cluster Labels'] == 1, all_toronto_merged.columns[[1] + list(range(5, all_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Central Toronto,1,Park,Women's Store,Distribution Center,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
45,North York,1,Park,Women's Store,Distribution Center,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [303]:
all_toronto_merged.loc[all_toronto_merged['Cluster Labels'] == 2, all_toronto_merged.columns[[1] + list(range(5, all_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,North York,2,Trail,Construction & Landscaping,Park,Women's Store,Ethiopian Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room
26,Scarborough,2,Trail,Ethiopian Restaurant,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Women's Store,Discount Store
49,North York,2,Bakery,Park,Trail,Women's Store,Ethiopian Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room


In [304]:
all_toronto_merged.loc[all_toronto_merged['Cluster Labels'] == 3, all_toronto_merged.columns[[1] + list(range(5, all_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
103,Canadian postal codes,3,Coffee Shop,Hotel,Café,Restaurant,Asian Restaurant,Japanese Restaurant,Concert Hall,Gym,Bar,Sushi Restaurant
78,Scarborough,3,Chinese Restaurant,Discount Store,Department Store,Shopping Mall,Bubble Tea Shop,Supermarket,Sushi Restaurant,Bakery,Badminton Court,Skating Rink
93,Etobicoke,3,Performing Arts Venue,Gym,Convenience Store,Pub,Women's Store,Ethiopian Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School
55,North York,3,Italian Restaurant,Coffee Shop,Sandwich Place,Pharmacy,Indian Restaurant,Cosmetics Shop,Pub,Restaurant,Café,Butcher
20,Downtown Toronto,3,Coffee Shop,Seafood Restaurant,Cocktail Bar,Pharmacy,Cheese Shop,Farmers Market,Bakery,Restaurant,Breakfast Spot,Beer Bar
58,Scarborough,3,College Stadium,Café,General Entertainment,Skating Rink,Escape Room,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant
43,West Toronto,3,Bar,Café,Coffee Shop,Restaurant,Sandwich Place,Gift Shop,Arts & Crafts Store,Bakery,Breakfast Spot,French Restaurant
100,East Toronto,3,Coffee Shop,Hotel,Café,Restaurant,Asian Restaurant,Japanese Restaurant,Concert Hall,Gym,Bar,Sushi Restaurant
87,Downtown Toronto,3,Italian Restaurant,Coffee Shop,Gym / Fitness Center,Café,Park,Bar,French Restaurant,Speakeasy,Gym,Lounge
21,York,3,Café,Gym,Park,Sporting Goods Shop,Mexican Restaurant,Bakery,Beer Store,Women's Store,Escape Room,Eastern European Restaurant


In [305]:
all_toronto_merged.loc[all_toronto_merged['Cluster Labels'] == 4, all_toronto_merged.columns[[1] + list(range(5, all_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,North York,4,Business Service,Women's Store,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
